In [8]:
import sys,os
current_dir = os.getcwd()
sys.path.append(os.path.dirname(os.path.abspath(current_dir)))

In [9]:
from Model_Vocals import Model_Vocals
model = Model_Vocals()
#model.SaveModel()
model.initialize(3)

LoadModel...
LoadModel...DONE


In [12]:
audio_path = os.path.join(current_dir, "곰세마리.wav")
label = "곰 세마리가 한 집에 있어 아빠곰 엄마곰 애기곰 아빠곰은 뚱뚱해 엄마 곰은 날씬해 애기곰은 너무 귀여워 으쓱으쓱 잘한다."

result = model(audio_path, label)

infer .. start
infer .. Done
origintext: ㄱ predtext: ㄱ
origintext: ㅗ predtext: ㅗㅇㅗ
origintext: ㅁ predtext: ㅁ
origintext: ㅅ predtext: ㅅ
origintext: ㅔ predtext: ㅐㅇㅐ
origintext: ㅁ predtext: ㅁ
origintext: ㅏ predtext: ㅏㅇㅏ
origintext: ㄹ predtext: ㄹ
origintext: ㅣ predtext: ㅣㅇㅣ
origintext: ㄱ predtext: ㄱ
origintext: ㅏ predtext: ㅏㅇㅏ
origintext: ㅎ predtext: ㅎ
origintext: ㅏ predtext: ㅏㅇㅏ
origintext: ㄴ predtext: ㄴ
origintext: ㅈ predtext: ㅈ
origintext: ㅣ predtext: ㅣ
origintext: ㅂ predtext: ㅂ
origintext: ㅔ predtext: ㅔㅇㅔ
origintext: ㅇ predtext: ㅇ
origintext: ㅣ predtext: ㅣ
origintext: ㅆ predtext: ㅆ
origintext: ㅓ predtext: ㅓㅇㅓ
origintext: ㅇ predtext: ㅇ
origintext: ㅏ predtext: ㅏ
origintext: ㅃ predtext: ㅃ
origintext: ㅏ predtext: ㅏㅇㅏ
origintext: ㄱ predtext: ㄱ
origintext: ㅗ predtext: ㅗㅇㅗ
origintext: ㅁ predtext: ㅁ
origintext: ㅇ predtext: ㅇ
origintext: ㅓ predtext: ㅓ
origintext: ㅁ predtext: ㅁ
origintext: ㅁ predtext: ㅁ
origintext: ㅏ predtext: ㅏㅇㅏ
origintext: ㄱ predtext: ㄱ
origintext: ㅗ predtext: ㅗㅇㅗ
origintext:

In [13]:
from utils.Audio import GetAudio
audio =GetAudio(audio_path)
display(audio)
for w in result:
    start = w['start']
    end = w['end']
    print(w['origin'],w['start'],w['end'])
    display(audio[start*1000:end*1000])

곰 1.28 1.7


세 1.78 1.96


마 2.02 2.14


리 2.3 2.44


가 2.84 2.96


...

으 23.296458333333334 23.616458333333334


쓱 24.096458333333334 24.396458333333335


잘 24.596458333333334 25.116458333333334


한 25.076458333333335 25.516458333333333


다 25.596458333333334 25.676458333333336


In [5]:
model.close_pool()  